In [2]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

## RDD 생성하기
데이터 헤더: "사용일자","노선명","역ID","역명",승차총승객수,하차총승객수

In [3]:
_sub=["20150101,2호선,0236,영등포구청,6199,6219",
"20150101,2호선,0237,당산,7982,8946",
"20150101,2호선,0238,합정,17406,15241",
"20150101,3호선,0309,지축,515,538",
"20150101,3호선,0310,구파발,6879,6260",
"20150101,3호선,0311,연신내,20031,19470",
"20150101,3호선,0312,불광,9519,11029",
"20150101,4호선,0425,회현,7465,7574",
"20150101,4호선,0426,서울역,3943,10823",
"20150101,경부선,1002,남영,4340,4535",
"20150101,경부선,1003,용산,28980,27684",
"20150101,경부선,1004,노량진,23021,23862",
"20150101,경부선,1005,대방,6360,6476",
"20150101,경부선,1006,영등포,37247,36102",
"20150101,경원선,1008,이촌,1940,1507",
"20150101,경원선,1009,서빙고,911,1000",
"20150101,경원선,1010,한남,1885,1863",
"20150101,경원선,1011,옥수,43,37"]

In [4]:
_subRdd = spark.sparkContext.parallelize(_sub)

In [5]:
for i in _subRdd.take(1):
    print (i)

20150101,2호선,0236,영등포구청,6199,6219


## 데이터 정리
- `map()`을 사용해서 ,를 제외

In [6]:
for i in _subRdd.map(lambda x:x.split(',')).collect():
    for j in i:
        print (j, end=" ")
    print()

20150101 2호선 0236 영등포구청 6199 6219 
20150101 2호선 0237 당산 7982 8946 
20150101 2호선 0238 합정 17406 15241 
20150101 3호선 0309 지축 515 538 
20150101 3호선 0310 구파발 6879 6260 
20150101 3호선 0311 연신내 20031 19470 
20150101 3호선 0312 불광 9519 11029 
20150101 4호선 0425 회현 7465 7574 
20150101 4호선 0426 서울역 3943 10823 
20150101 경부선 1002 남영 4340 4535 
20150101 경부선 1003 용산 28980 27684 
20150101 경부선 1004 노량진 23021 23862 
20150101 경부선 1005 대방 6360 6476 
20150101 경부선 1006 영등포 37247 36102 
20150101 경원선 1008 이촌 1940 1507 
20150101 경원선 1009 서빙고 911 1000 
20150101 경원선 1010 한남 1885 1863 
20150101 경원선 1011 옥수 43 37 


호선별 데이터 개수를 출력해 보자
- map()의 3번째 인덱스는 앞에서부터 4번째 철자를 출력한다.
- 컴마로 분리해서 인덱스를 사용해야 제대로 데이터를 출력한다.

In [7]:
_subRdd.map(lambda x:int(x[3])).collect()

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]

In [8]:
_subRdd.map(lambda x:x.split(',')).map(lambda x:int(x[4])).collect()

[6199,
 7982,
 17406,
 515,
 6879,
 20031,
 9519,
 7465,
 3943,
 4340,
 28980,
 23021,
 6360,
 37247,
 1940,
 911,
 1885,
 43]

## 총 승차인원의 합계를 reduce()로 합산한다.
- reduce()함수는 인자를 2개 받아서 합계를 구한다.

In [9]:
_subRdd.map(lambda x:x.split(',')).map(lambda x:(int(x[4]))).reduce(lambda x,y:x+y)

184666

## 호선별 승차인원 평균을 계산한다.
Pair RDD를 사용하므로, 사전에 데이터를 (호선, 승차인원) 형식으로 구성한다.

In [11]:
_subLineByPassengers=_subRdd.map(lambda x:x.split(',')).map(lambda x: (x[1],int(x[4])))
sum_counts = _subLineByPassengers.combineByKey(
    (lambda x: (x, 1)), # the initial value, with value x and count 1
    (lambda acc, value: (acc[0]+value, acc[1]+1)), # how to combine a pair value with the accumulator: sum value, and increment count
    (lambda acc1, acc2: (acc1[0]+acc2[0], acc1[1]+acc2[1])) # combine accumulators
)

In [12]:
for i in sum_counts.collect():
    for each in i:
        print (each, end=" ")
    print()

2호선 (31587, 3) 
3호선 (36944, 4) 
4호선 (11408, 2) 
경부선 (99948, 5) 
경원선 (4779, 4) 


In [13]:
for i in sum_counts.collect():
    print (i[0],i[1][0],i[1][1])

2호선 31587 3
3호선 36944 4
4호선 11408 2
경부선 99948 5
경원선 4779 4


#### 평균

In [14]:
averageByKey = sum_counts.map(lambda x: (x[0],x[1][0]/x[1][1]))
averageByKey.collect()

[('2호선', 10529.0),
 ('3호선', 9236.0),
 ('4호선', 5704.0),
 ('경부선', 19989.6),
 ('경원선', 1194.75)]